In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from glob import glob


from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Flatten
import keras
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

import os
print(os.listdir('../input'))
#install keras-efficientnet



Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

['install', 'aptos2019-blindness-detection', 'weights']


In [2]:
!pip install -U '../input/install/efficientnet-0.0.3-py2.py3-none-any.whl'

Processing /kaggle/input/install/efficientnet-0.0.3-py2.py3-none-any.whl


In [3]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def circle_crop(img):   
    #img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img

In [4]:
df=pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
l1=[s+'.png' for s in df['id_code']]
df['id_code']=l1
l2=[str(s) for s in df['diagnosis']]
df['diagnosis']=l2
#print(df)
sigmaX=10
IMG_SIZE=228
def preprocess(img_name):
    
    image = cv2.imread(img_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = circle_crop(image)
    image=cv2.addWeighted (image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return(image)

BATCH_SIZE = 32
TRAIN_IMG_PATH='../input/aptos2019-blindness-detection/train_images'
# Add Image augmentation to our generator
train_datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,zca_whitening=True)

# Use the dataframe to define train and validation generators
train_generator = train_datagen.flow_from_dataframe(df, 
                                                    x_col='id_code', 
                                                    y_col='diagnosis',
                                                    directory = TRAIN_IMG_PATH,
                                                    target_size=(IMG_SIZE,IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='sparse',
                                                    preprocessing_function=preprocess, 
                                                    subset='training')

/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 3662 validated image filenames belonging to 5 classes.


In [5]:
#Use Efficient Net
from efficientnet import EfficientNetB5
efficient_model=EfficientNetB5(weights=None,input_shape=(IMG_SIZE,IMG_SIZE,3),include_top=False)
efficient_model.load_weights('../input/weights/efficientnet-b5_imagenet_1000_notop.h5')

In [6]:
model=Sequential()
model.add(efficient_model)
model.add(layers.Flatten())
model.add(layers.Dense(2048))
model.add(layers.LeakyReLU())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(5,activation='softmax'))

model.layers[0].trainable = False

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

"""
for l in model.layers:
    print(l.name, l.trainable)
"""

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b5 (Model)      (None, 8, 8, 2048)        28513520  
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              268437504 
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 10245     
Total params: 296,961,269
Trainable params: 268,447,749
Non-trainable params: 28,513,520
_____________________________________________________

In [7]:
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=20)

Epoch 1/20


/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


114/114 [==============================] - 423s 4s/step - loss: 6.1669 - acc: 0.6105
Epoch 2/20
114/114 [==============================] - 407s 4s/step - loss: 5.5480 - acc: 0.6548
Epoch 3/20
114/114 [==============================] - 417s 4s/step - loss: 6.0294 - acc: 0.6248
Epoch 4/20
114/114 [==============================] - 420s 4s/step - loss: 6.8811 - acc: 0.5722
Epoch 5/20
114/114 [==============================] - 416s 4s/step - loss: 6.5435 - acc: 0.5937
Epoch 6/20
114/114 [==============================] - 406s 4s/step - loss: 7.7933 - acc: 0.5163
Epoch 7/20
114/114 [==============================] - 405s 4s/step - loss: 5.8498 - acc: 0.6369
Epoch 8/20
114/114 [==============================] - 404s 4s/step - loss: 5.9161 - acc: 0.6328
Epoch 9/20
114/114 [==============================] - 406s 4s/step - loss: 6.1188 - acc: 0.6202
Epoch 10/20
114/114 [==============================] - 403s 4s/step - loss: 5.6953 - acc: 0.6459
Epoch 11/20
114/114 [=============================

In [8]:
from keras.utils import to_categorical
#X_test = ImageDataset(csv_file = '../input/aptos2019-blindness-detection/test.csv', root_dir = '../input/aptos2019-blindness-detection/test_images/')
df_test=pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
l1=[s+'.png' for s in df_test['id_code']]
df_test['id_code']=l1
#print(df_test)
datagen=ImageDataGenerator()
generator = datagen.flow_from_dataframe(df_test,x_col='id_code',
                                        directory='../input/aptos2019-blindness-detection/test_images',
                                        target_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=1,
                                        class_mode=None,
                                        preprocessing_function=preprocess,
                                        shuffle=False)  

predictions = model.predict_generator(generator,1928)
"""TEST_IMG_PATH='../input/aptos2019-blindness-detection/test_images'
N = df_test.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.float32)
for i, image_id in enumerate(df_test['id_code']):
    x_test[i, :, :, :] = preprocess(f'{TEST_IMG_PATH}/{image_id}')"""


#print(np.argmax(predictions, axis=1))
submission = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
submission['diagnosis'] = np.argmax(predictions,axis=1)    
submission.to_csv('submission.csv', index=False, encoding='utf-8')

Found 1928 validated image filenames.
